### Using DVC checkpoints to train an MNIST classifier.

This notebook introduces users to DVC checkpoints. This example uses the [MNIST](http://yann.lecun.com/exdb/mnist/) data of handwritten digits and builds a classification model to predict the digit (0-9) in each image.

### Model script


The model is built in [keras](https://keras.io/) as a convolutional neural network with an architecture that is simple enough to run quickly with few resources.

Let's look at the model training script:

In [1]:
%%bash
cat train.py

import os
import yaml
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.random import set_seed

from dvc.api import make_checkpoint

num_classes = 10
input_shape = (28, 28, 1)
epochs = 10
batch_size = 128


def get_data():
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return (x_train, y_train), (x_test, y_test)

def get_model():
    set_seed(0)
    model= keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(4, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(8, kernel_size=(3, 3), activation="relu"),
        layers.Max

The script does the following:
1. Loads or creates the model (want to be able to load existing model to iteratively train from warm start).
2. Loads MNIST data, including transformations and train/test splits.
3. For each epoch, trains, evaluates, and saves.

To enable DVC checkpoints, the `make_checkpoint()` function is called after each epoch.

### DVC Pipeline

The last step to set up checkpoints is a `dvc.yaml` file to describe the the dependencies and outputs of the model training stage. Review the contents of `dvc.yaml`:

In [2]:
%%bash
cat dvc.yaml

stages:
  # Arbitraty name for model training stage
  train:
    # Command to execute
    cmd: python train.py
    # Dependencies
    deps:
    - train.py
    # Outputs
    outs:
    - model.tf:
        # Required for checkpoints
        checkpoint: true
    # Metrics
    metrics:
    - metrics.yaml:
        # Track with git instead of dvc
        cache: false


The `dvc.yaml` includes a single stage arbitrarily named `train` that executes the command `python train.py`. The `train.py` script is its only dependency. The model output is saved in the `model.tf` directory and metrics are saved to `metrics.yaml`.

For users familiar with DVC, nothing about this stage is unusual except for `checkpoint: true`, which tells DVC to treat `model.tf` differently from a typical output, since `train.py` also loads the previously trained model from there.

### Train the model

Run an experiment to start training the model. Even with a simple architecture, the model may take a few minutes to complete since it will train for 10 epochs.

**NOTE:** Make sure the repo is up to date in git (commit all changes) before running experiments.

In [3]:
%%bash
dvc exp run

Running stage 'train':
> python train.py
79/79 [==============================] - 1s 6ms/step - loss: 0.2801 - accuracy: 0.9324
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '7ed5b0c'.
79/79 [==============================] - 1s 6ms/step - loss: 0.1804 - accuracy: 0.9540
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '93c131a'.
79/79 [==============================] - 1s 7ms/step - loss: 0.1438 - accuracy: 0.9617
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'e8c1c50'.
79/79 [==============================] - 1s 6ms/step - loss: 0.1253 - accuracy: 0.9670
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'da61bb6'.
79/79 [==============================] - 1s 6ms/step - loss: 0.1131 - accuracy: 0.9689
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '07c07fb'.
79/79 [==============================] - 1s 6ms/step - loss: 0.1044 - accuracy: 0.9713
Updating lock file 'dvc.lock'
Checkpoint

2021-02-03 19:55:28.192102: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-02-03 19:55:28.192128: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-02-03 19:55:30.127520: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-03 19:55:30.127706: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-02-03 19:55:30.127724: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-02-03 19:55:30.127748: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running 

Review the output of the run, including identifying hashes, metrics, and parameters:

In [4]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ Experiment    ┃ Created  ┃    acc ┃     loss ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ workspace     │ -        │ 0.9753 │ 0.088837 │
│ keras         │ 07:38 PM │      - │          │
│ │ ╓ exp-a969d │ 07:56 PM │ 0.9753 │ 0.088837 │
│ │ ╟ 7ec73ad   │ 07:56 PM │ 0.9748 │ 0.092141 │
│ │ ╟ 5c93ce3   │ 07:56 PM │ 0.9745 │  0.09462 │
│ │ ╟ 73bcdaf   │ 07:56 PM │  0.974 │  0.10015 │
│ │ ╟ 9b2855d   │ 07:56 PM │ 0.9713 │   0.1044 │
│ │ ╟ 07c07fb   │ 07:56 PM │ 0.9689 │  0.11313 │
│ │ ╟ da61bb6   │ 07:56 PM │  0.967 │   0.1253 │
│ │ ╟ e8c1c50   │ 07:55 PM │ 0.9617 │  0.14378 │
│ │ ╟ 93c131a   │ 07:55 PM │  0.954 │  0.18039 │
│ ├─╨ 7ed5b0c   │ 07:55 PM │ 0.9324 │  0.28014 │
└───────────────┴──────────┴────────┴──────────┘


`exp-504ba` was created in the `keras` branch, and a checkpoint was generated for each of the 10 epochs when `make_checkpoint()` was called.

The results from the final epoch are present in the workspace, which can be confirmed either manually or using `dvc metrics show`:

In [5]:
%%bash
cat metrics.yaml

acc: 0.9753000140190125
loss: 0.08883664757013321


In [6]:
%%bash
dvc metrics show

Path          acc     loss
metrics.yaml  0.9753  0.08884


### Adding checkpoints

Run the experiment again to continue training:

In [7]:
%%bash
dvc exp run

Modified checkpoint experiment based on 'exp-a969d' will be created
Running stage 'train':
> python train.py
79/79 [==============================] - 1s 6ms/step - loss: 0.0831 - accuracy: 0.9765
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '86091fb'.
79/79 [==============================] - 0s 6ms/step - loss: 0.0810 - accuracy: 0.9782
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'ab509db'.
79/79 [==============================] - 1s 6ms/step - loss: 0.0816 - accuracy: 0.9765
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '2647336'.
79/79 [==============================] - 0s 6ms/step - loss: 0.0818 - accuracy: 0.9777
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '80879be'.
79/79 [==============================] - 1s 6ms/step - loss: 0.0766 - accuracy: 0.9786
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'aa5a438'.
79/79 [==============================] - 1s 6ms/step - loss: 0.0749 - accuracy: 0.9795
Upda

2021-02-03 19:58:10.120392: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-02-03 19:58:10.120433: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-02-03 19:58:12.407555: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-03 19:58:12.407759: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-02-03 19:58:12.407776: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-02-03 19:58:12.407815: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running 

In [8]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ Experiment            ┃ Created  ┃    acc ┃     loss ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ workspace             │ -        │   0.98 │ 0.071565 │
│ keras                 │ 07:38 PM │      - │          │
│ │ ╓ exp-96de6         │ 07:59 PM │   0.98 │ 0.071565 │
│ │ ╟ e18103c           │ 07:59 PM │ 0.9762 │ 0.076957 │
│ │ ╟ 8d0bde1           │ 07:59 PM │  0.979 │ 0.075905 │
│ │ ╟ 4a4e212           │ 07:59 PM │ 0.9788 │ 0.075882 │
│ │ ╟ 674e213           │ 07:59 PM │ 0.9795 │ 0.074922 │
│ │ ╟ aa5a438           │ 07:58 PM │ 0.9786 │ 0.076629 │
│ │ ╟ 80879be           │ 07:58 PM │ 0.9777 │ 0.081824 │
│ │ ╟ 2647336           │ 07:58 PM │ 0.9765 │ 0.081557 │
│ │ ╟ ab509db           │ 07:58 PM │ 0.9782 │ 0.081043 │
│ │ ╟ 86091fb (efa7315) │ 07:58 PM │ 0.9765 │ 0.083118 │
│ │ ╓ efa7315           │ -        │      - │          │
│ │ ╟ 7ec73ad           │ -        │      - │          │
│ │ ╟ 5c93ce3           │ -    